# Mesh generation for tutorial 05

In [ ]:
import numpy as np
import pygmsh
from mpi4py import MPI
from dolfinx import cpp
from dolfinx.mesh import create_mesh, create_meshtags
from dolfinx.io import ufl_mesh_from_gmsh, XDMFFile
from dolfinx.plotting import plot

In [ ]:
assert MPI.COMM_WORLD.size == 1, "This mesh generation notebook is supposed to be run in serial"

### Geometrical parameters

In [ ]:
r = 3
lcar = 1. / 4.

### Create pygmsh mesh

In [ ]:
geom = pygmsh.geo.Geometry()
c0 = geom.add_circle([0.0, 0.0, 0.0], r, lcar)
geom.add_physical(c0.plane_surface, label=0)
geom.add_physical(c0.line_loop.lines, label=1)
pygmsh_mesh = pygmsh.generate_mesh(geom)

### Convert to a dolfinx mesh

In [ ]:
cells, x = pygmsh_mesh.cells_dict["triangle"], pygmsh_mesh.points[:, :2]
mesh = create_mesh(MPI.COMM_WORLD, cells, x, ufl_mesh_from_gmsh("triangle", x.shape[1]))

In [ ]:
plot(mesh)

### Extract boundaries as MeshTags

In [ ]:
boundaries_entities = pygmsh_mesh.cells_dict["line"]
boundaries_values = pygmsh_mesh.cell_data_dict["gmsh:physical"]["line"]
mesh.topology.create_connectivity(mesh.topology.dim - 1, 0)
boundaries = create_meshtags(mesh, mesh.topology.dim - 1,
                             cpp.graph.AdjacencyList_int32(boundaries_entities), np.int32(boundaries_values))
boundaries.name = "boundaries"

### Save mesh and boundaries

In [ ]:
with XDMFFile(MPI.COMM_WORLD, "circle.xdmf", "w") as file:
    file.write_mesh(mesh)
    mesh.topology.create_connectivity(mesh.topology.dim - 1, mesh.topology.dim)
    file.write_meshtags(boundaries)